In [11]:
import pprint
import pathlib
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans




In [30]:
def read_csv_data(df):
    """
    Reads two csv files (tweets and events) and 
    returns a pandas dataframe for each file
    """
    return pd.read_csv(df, header=0)

In [31]:
syria_data_dir = pathlib.Path('/Users/adamstueckrath/Desktop/syria_data/')
results_csv = syria_data_dir / 'model' / 'model_data' / 'tweet_modelvtest.csv'
results_df = read_csv_data(results_csv)
print(results_df.dtypes)
print(results_df.shape)

results_df = results_df.dropna(subset=['cosine_value'])
print(results_df.shape)

results_df = results_df[(results_df[['cosine_value']] != 0).all(axis=1)]
print(results_df.shape)

/Users/adamstueckrath/.pyenv/versions/3.6.5/envs/syria-project-3.6.5/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3183: DtypeWarning: Columns (9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


tweet_id                     object
tweet_created_at             object
tweet_text                   object
tweet_sentiment_compound    float64
tweet_sentiment_score       float64
tweet_sentiment_label        object
tweet_text_normalize         object
event_id                    float64
cosine_value                float64
event_date                   object
location                     object
event_text                   object
event_text_normalize         object
dtype: object
(638032, 13)
(106720, 13)
(68277, 13)


In [32]:
def remove_tweets_filter(df, filter_amount=None):
    """
    Drops Nan values from the tweet_text_normalize column and 
    returns dataframe filtered or whole
    """
    df = df.dropna(subset=['tweet_text_normalize'])
    if filter_amount:
        df = df[:filter_amount]

    return df

def eval_dataframe(df):
    """
    Evaluates dataframe column saved as string format 
    into a python list object
    """
    df['tweet_text_normalize'] = df['tweet_text_normalize'].apply(eval)
    df['event_text_normalize'] = df['event_text_normalize'].apply(eval)
    return df

def create_corpus(df):
    """
    Creates two corpus' for the algorithm. The tweet corpus
    will be the training set and the event corpus will be the query.
    """
    # column to list
    tweet_normalize_list = df.tweet_text_normalize.tolist()
    event_normalize_list = df.event_text_normalize.tolist()
    # join processed text together for corpus'
    tweet_corpus = [ ' '.join(x) for x in tweet_normalize_list ]
    event_corpus = [ ' '.join(x) for x in event_normalize_list ]

    return tweet_corpus, event_corpus


In [33]:
results_df = remove_tweets_filter(results_df)
results_df = eval_dataframe(results_df)
tweet_corp, event_corp = create_corpus(results_df)

print(results_df.shape)
print(len(tweet_corp), len(event_corp))

(68277, 13)
68277 68277


In [41]:
def tfidf_algo(tweet_corpus, print_shape=False):
    """
    returns a two document term matrices of document (y) and featured words (x)

    fit_transform(): Learn vocabulary and idf, return term-document matrix.
    transform(): Transform documents to document-term matrix. Uses the vocabulary and 
    document frequencies (df) learned by fit_transform()
    """
    vectorizer = TfidfVectorizer(min_df=.000025, max_features=10000)
    tweetVectorizerArray = vectorizer.fit_transform(tweet_corpus)
    print(vectorizer.get_feature_names())
    if print_shape:
        print('Tweets: {} Events: {}'.format(tweetVectorizerArray.shape, eventVectorizerArray.shape))
    return tweetVectorizerArray


In [42]:
tweetVectorizerArray= tfidf_algo(tweet_corp)
print(tweetVectorizerArray.get_feature_names())
# tweet_corp, event_corp
# silhouette_score(tweetVectorizerArray.toarray(), eventVectorizerArray.toarray(), sample_size=50)

['00', '000', '00000000', '000s', '00588c', '01', '02', '03', '039', '04', '05', '06', '06ami', '07', '08', '09', '0900', '0e', '10', '100', '1000', '100k', '101', '101st', '109', '10m', '10pm', '10th', '10tv', '11', '110', '114', '11678', '119', '11th', '12', '120', '123', '124th', '12th', '13', '14', '140', '14ns', '14th', '15', '150', '150k', '158', '15th', '16', '1600', '16th', '17', '1776', '18', '180', '185', '18th', '19', '1900', '1917', '1920', '1941', '1945', '1949', '1951', '1957', '1959', '1960', '1960s', '1967', '1968', '1970', '1970s', '1972', '1980', '1983', '1984', '1986', '1990', '1991', '1997', '1998', '19th', '1a', '1ab', '1b', '1m', '1st', '1yr', '20', '200', '2000', '200000', '2001', '2003', '2006', '2007', '2008', '2009', '201', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2017syria', '2018', '2020', '2032', '20th', '21', '215', '21pm', '21st', '21wire', '22', '220', '222k', '224', '226', '226bn', '23', '230', '2305', '234radio', '23rd', '24', '

AttributeError: get_feature_names not found

In [23]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2).fit(eventVectorizerArray)
data2D = pca.transform(eventVectorizerArray)  

In [25]:
for num_clusters in range(2,3):
    km = KMeans(n_clusters=num_clusters,
            n_init=10,                        # number of iterations with different seeds
            random_state=1                    # fixes the seed 
           )
    cluster_labels = km.fit_predict(data2D)
    print(cluster_labels)
    silhouette_avg = silhouette_score(data2D, cluster_labels)
    print(silhouette_avg)

[1 1 1 ... 0 1 1]


KeyboardInterrupt: 

In [ ]:
for n_clusters in range(2, 10):
    km = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=100, n_init=1)
    cluster_labels = km.fit_predict(eventVectorizerArray)
    silhouette_avg = silhouette_score(eventVectorizerArray, cluster_labels, sample_size=1000)

    print(n_clusters, silhouette_avg)